In [1]:
import numpy as np
from jetbot import Robot
from jetbot import Camera
import cv2
print(cv2.__version__)
import asyncio
from PIL import Image
from IPython.display import clear_output
import matplotlib.pyplot as plt
import ipywidgets.widgets as widgets
import traitlets
from jetbot import bgr8_to_jpeg

4.1.1


In [2]:
robot = Robot()
#camera = Camera.instance(width=1280, height=720)
#camera = Camera.instance(width=1024,height=800)
#camera = Camera.instance()
#net = cv2.dnn.readNet('data/one_sphere/yolov3.weights','data/one_sphere/yolov3.cfg' )

In [3]:
from datetime import datetime

In [4]:
from PIL import Image

In [5]:
# print(camera)

In [6]:
#!pip3 install imutils

In [114]:
from imutils.video import FileVideoStream, FPS
from queue import Queue
from threading import Thread
import time
import sys

from threading import Thread
import cv2, time

class ThreadedCamera:
    def __init__(self, path, path_out):
        self.capture = cv2.VideoCapture(path)
        self.capture.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    
        # FPS = 1/X
        # X = desired FPS
#         self.FPS = 1/40
        self.FPS = 1/ int((self.capture.get(cv2.CAP_PROP_FPS)))
        print("FPS delay: " + str(self.FPS))
        print("Get FPS value: " + str(int(self.capture.get(cv2.CAP_PROP_FPS))))
        self.FPS_MS = int(self.FPS * 1000)
        
        self.previous_frame = None
        self.fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        #Hardcoded width and height values, as those are being read only once the frame is captured
        self.output_video = cv2.VideoWriter(path_out, self.fourcc, int(self.capture.get(cv2.CAP_PROP_FPS))/18, (1280,720))
        self.availableFrames = True
        self.initialized = False
        # Start frame retrieval thread
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

        
        
    def update(self):
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
                if self.initialized == False:
                    self.previous_frame = self.frame
                    self.initialized=True
                if self.status == False:
                    self.availableFrames = False
                    sys.exit()
                time.sleep(self.FPS)
#                 print("self.capture is opened(): " + str(time.time()))
            
    def get_frame(self):
        input_image = cv2.cvtColor(self.frame, cv2.COLOR_RGBA2RGB)
        h,w,c = input_image.shape
        print("Image input size: " + str(h) + ", " + str(w) + ", c: " + str(c))
        blob = cv2.dnn.blobFromImage(image=cv2.resize(input_image,(w,h)), scalefactor=1.0, size=(300,300), swapRB=True)
        net.setInput(blob)
        detections = net.forward()
        for j in np.arange(0, detections.shape[2]):
            confidence = detections[0,0,j,2]
            if confidence > 0.70:
                print(confidence)
                idx = int(detections[0,0,j,1])
                box = detections[0,0,j,3:7] * np.array([w,h,w,h])
                (startX, startY, endX, endY) = box.astype("int")
                print("Id: " + str(idx))
                label = "{}: {:.2f}%".format(CLASSES[idx-1], confidence*100)
                print(label)
                cv2.rectangle(input_image, (startX,startY), (endX,endY), COLORS[idx-1],2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(input_image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx-1],2)
                print("Detected " + str(CLASSES[idx-1]) +  " at X position: " + str(startX))
                #starting x of the detected object on the right side
                print("startX " + str(startX))
                print("w" + str(w/2))
        difference = cv2.subtract(self.previous_frame, input_image)
        if np.any(difference) and self.initialized:
            self.output_video.write(input_image)
        self.previous_frame = input_image
#         _, encoded_image = cv2.imencode('.png', input_image)
#         bytes_img_obj = encoded_image.tobytes()
#         gt_widget = widgets.Image(value=bytes_img_obj, format="png",width=300, height=300)
#         camera_link = traitlets.dlink((gt_widget,'value'), (image_trait, 'value'))
#         if(self.initialized == False):
#     #         clear_output(wait=True)
#             display(image_trait)
#             self.initialized = True
#         print(time.time())

In [115]:
from jetbot import Robot
import ipywidgets.widgets as widgets

robot = Robot()
import numpy as np
import cv2
CLASSES = ["sphere","cube","cylinder","cuboid"]

COLORS = np.random.uniform(0,255,size=(len(CLASSES),3))

net = cv2.dnn.readNetFromTensorflow('data/07_corridor/3vers/frozen_inference_graph.pb', 'data/07_corridor/3vers/final_graph.pbtxt')

VIDEO_NAME = 'test_beg'
VIDEO_TO_PROCESS = 'output_video/' + VIDEO_NAME + '.mp4'
VIDEO_TO_OUTPUT = 'output_video/processed_videos/'+VIDEO_NAME + '.avi'

import cv2
import matplotlib.pyplot as plt
image = 0
# cap = cv2.VideoCapture(VIDEO_TO_PROCESS)

In [116]:
fvs = ThreadedCamera(VIDEO_TO_PROCESS, VIDEO_TO_OUTPUT)

image_trait = widgets.Image(format='png', width=300, height=300)

i=0
fps = FPS().start()
while fvs.availableFrames:
    try:
        fvs.get_frame()
    except AttributeError:
        pass
    except AssertionError:
        pass
fps.stop()
print("INFO: Elapsed time: {:.2f}".format(fps.elapsed()))
print("INFO: Approx FPS: {:.2f}".format(fps.fps()))
# fvs.stop()
fvs.capture.release()
fvs.output_video.release()

FPS delay: 0.016666666666666666
Get FPS value: 60
Image input size: 720, 1280, c: 3
0.88534343
Id: 1
sphere: 88.53%
Detected sphere at X position: 751
startX 751
w640.0
Image input size: 720, 1280, c: 3
0.8969203
Id: 3
cylinder: 89.69%
Detected cylinder at X position: 624
startX 624
w640.0
Image input size: 720, 1280, c: 3
0.82091194
Id: 3
cylinder: 82.09%
Detected cylinder at X position: 454
startX 454
w640.0
Image input size: 720, 1280, c: 3
0.7688311
Id: 1
sphere: 76.88%
Detected sphere at X position: 523
startX 523
w640.0
0.8814485
Id: 3
cylinder: 88.14%
Detected cylinder at X position: 446
startX 446
w640.0
Image input size: 720, 1280, c: 3
Image input size: 720, 1280, c: 3
0.82443905
Id: 1
sphere: 82.44%
Detected sphere at X position: 526
startX 526
w640.0
0.8496297
Id: 3
cylinder: 84.96%
Detected cylinder at X position: 447
startX 447
w640.0
Image input size: 720, 1280, c: 3
Image input size: 720, 1280, c: 3
0.72978485
Id: 1
sphere: 72.98%
Detected sphere at X position: 782
star

In [28]:
robot.stop()
# cap.release()
fvs.stop()
fps.stop()

AttributeError: 'ThreadedCamera' object has no attribute 'stop'

Main changes applied so far: changing the layer batch optimazation FusedBatchNormV3 to FusedBatchNorm because of incongruences in tensorflow module version